# This notebook is an example notebook to inference the latent diffusion model.

In [ ]:
import os
import random
import sys

import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision.transforms as T
from omegaconf import OmegaConf
from PIL import Image, ImageDraw

sys.path.append(os.getcwd() + "/ldm")
from ldm.models.diffusion.ddim import DDIMSampler
from ldm.util import instantiate_from_config

transform_PIL = T.ToPILImage()

In [ ]:
# config
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

print(f"Using device: {device}")
yaml_path = "ldm/models/ldm/inpainting_big/config.yaml"
# model_path = "ldm/models/ldm/inpainting_big/last.ckpt"
model_path = "logs/checkpoints/last-v2.ckpt"

In [ ]:
# create model


def create_model(device):

    # load config and checkpoint
    config = OmegaConf.load(yaml_path)
    config.model["params"]["ckpt_path"] = model_path

    model = instantiate_from_config(config.model)
    sampler = DDIMSampler(model)
    model = model.to(device)

    return model, sampler

In [ ]:
def create_masked_image(image, mask):
    # Create a copy of the original image
    masked_image = image.copy()

    # Apply the mask to the original image
    masked_image.paste(Image.new("RGB", image.size, (0, 0, 0)), mask=mask)

    return masked_image


def random_rectangle_mask(image):
    # Create a copy of the original image
    masked_image = image.copy()
    image_x, image_y = image.size

    width = random.randint(70, 150)
    height = int(width * random.uniform(0.7, 1.5))

    # Create a black background mask
    mask = Image.new("L", (image_x, image_y), 0)
    draw = ImageDraw.Draw(mask)

    # Randomly generate the coordinates for the top-left and bottom-right corners of the rectangle
    x1 = random.randint(0, image.size[0] - width)
    y1 = random.randint(0, image.size[1] - height)
    x2 = x1 + width
    y2 = y1 + height

    # Draw a white rectangle on the mask
    draw.rectangle([x1, y1, x2, y2], fill=255)

    # Apply the mask to the original image
    create_masked_image(image, mask)

    return image, mask, masked_image

In [ ]:
def process_data(image, mask):

    # creating a 3 dimensional mask
    mask = np.array(mask)
    mask = np.expand_dims(mask, axis=2)

    # normalzie and transform the image into tensor
    image = np.array(image.convert("RGB"))
    image = image.astype(np.float32) / 255.0  #
    image = image[None].transpose(0, 3, 1, 2)
    image = torch.from_numpy(image)

    # normalzie and transform the mask into tensor
    mask = mask.astype(np.float32) / 255.0  #
    mask[mask < 0.1] = 0
    mask[mask >= 0.1] = 1
    mask = mask[None].transpose(0, 3, 1, 2)

    # produce the masked image by subtraction
    mask = torch.from_numpy(mask)
    masked_image = (1 - mask) * image

    batch = {
        "image_tensor": image,
        "mask_tensor": mask,
        "masked_image_tensor": masked_image,
    }
    for k in batch:
        batch[k] = batch[k] * 2.0 - 1.0

    return batch

In [ ]:
model, sampler = create_model(device)

In [ ]:
image = Image.open("./splits/test/images/20230707_005855_CAM_MAC1_5_BT.jpg").resize(
    (512, 512)
)

mask = (
    Image.open(
        "./splits/test/masks/20231106_175846_CAM_MAC1_5_BT.jpg",
    )
    .resize((512, 512))
    .convert("L")
)

In [ ]:
# image, mask, masked_image = random_rectangle_mask(image)
masked_image = create_masked_image(image, mask)

In [ ]:
fig, axes = plt.subplots(
    1, 3, figsize=(12, 4)
)  # Create a 1x3 grid for displaying 3 images

# Display each image on a separate subplot
axes[0].imshow(image)
axes[0].set_title("image")

axes[1].imshow(mask)
axes[1].set_title("mask")

axes[2].imshow(masked_image)
axes[2].set_title("masked_image")

plt.show()

In [ ]:
# Inference

# convert PIL image into input Torch Tensor
batch = process_data(image, mask)
image_tensor = batch["image_tensor"]
mask_tensor = batch["mask_tensor"]
masked_image_tensor = batch["masked_image_tensor"]


# encode masked image and concat downsampled mask
c = model.cond_stage_model.encode(masked_image_tensor.to(device))

# the mask is frst being downsampled
cc = torch.nn.functional.interpolate(mask_tensor.to(device), size=c.shape[-2:])
# concat the masked image and downsampled mask
c = torch.cat((c, cc), dim=1)
shape = (c.shape[1] - 1,) + c.shape[2:]

# diffusion process
samples_ddim, _ = sampler.sample(
    S=50, conditioning=c, batch_size=c.shape[0], shape=shape, verbose=False
)

# decode the latent vector (output)
x_samples_ddim = model.decode_first_stage(samples_ddim)


# denormalize the output
predicted_image_clamped = torch.clamp((x_samples_ddim + 1.0) / 2.0, min=0.0, max=1.0)

In [ ]:
output_PIL = transform_PIL(predicted_image_clamped[0])

In [ ]:
fig, axes = plt.subplots(
    1, 3, figsize=(12, 4)
)  # Create a 1x3 grid for displaying 3 images

# Display each image on a separate subplot
axes[0].imshow(image)
axes[0].set_title("original image")

axes[1].imshow(mask)
axes[1].set_title("masked image")

axes[2].imshow(output_PIL)
axes[2].set_title("Inpainted Image")

plt.show()